<a href="https://colab.research.google.com/github/epistimi4/emotion_based_music_recomendation/blob/master/Preprocessing/Spotify_MusicInformationRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Collection and Music Information Retrieval from Spotify #

# Data Collection and Spotify Music Information Retrieval #

# P3 - Emotion and context-aware recommendation system #

## Import libraries and install dependencies ##

In [0]:
!pip install spotipy

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [0]:
import numpy as np
import json
import pandas as pd
import IPython.display
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #to access authorised Spotify data

## Preprocessing ##
### Data Collection ###
#### Read datasets ####

In [2]:
import glob
dpath = 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\'

onlyfiles = glob.glob(dpath+'*.xlsx')
print(len(onlyfiles))
print(onlyfiles)

16
['C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_1.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_10.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_11.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_12.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_13.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_14.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_15.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_16.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_2.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Preprocessing\\user_data\\user_3.xlsx', 'C:\\usersCode\\MScDataScience\\Multimodal analysis\\Prepro

In [7]:
udata_prev = pd.DataFrame()
for f in onlyfiles:
  udata = pd.read_excel(f)
  final_data = pd.concat([udata_prev,udata], axis=0, ignore_index=True)
  udata_prev = final_data  

final_data.rename(columns = {
  'Id'  :'user_id'}, inplace=True)
print('# songs:',len(final_data))
final_data.tail(32)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# songs: 703


,user_id,activity,age,artist,gender,location,mood,period,song,song-artist
671,9,amusing,40,AC DC,male,home,peaceful,night,You shook me all night long,AC DC - You shook me all night long
672,9,amusing,40,Iron Maiden,male,home,calm,morning,Fear of the dark,Iron Maiden - Fear of the dark
673,9,amusing,40,Iron Maiden,male,home,calm,morning,Hallowed be thy name,Iron Maiden - Hallowed be thy name
674,9,amusing,40,Iron Maiden,male,home,calm,morning,Wasted years,Iron Maiden - Wasted years
675,9,amusing,40,Iron Maiden,male,home,calm,morning,Aces high,Iron Maiden - Aces high
676,9,amusing,40,Iron Maiden,male,home,calm,morning,Run to the hills,Iron Maiden - Run to the hills
677,9,amusing,40,Iron Maiden,male,home,calm,morning,The trooper,Iron Maiden - The trooper
678,9,amusing,40,Iron Maiden,male,home,calm,morning,2 minutes to midnight,Iron Maiden - 2 minutes to midnight
679,9,amusing,40,Metallica,male,home,calm,morning,Enter sandman,Metallica - Enter sandman
680,9,amusing,40,Metallica,male,home,calm,morning,Sad but true,Metallica - Sad but true


In [0]:
final_data['Unnamed: 10'].value_counts()

 Om                 1
 Venetian Snares    1
Name: Unnamed: 10, dtype: int64

### Authorization flows ###

### Using spotipy ###

#### Authorize app ####

In [0]:
client_id = '8038c4be0f8a4844982edce1b2282ebc'
client_secret = '1bd8da3a234e4060a6801a432c0e877d'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

### Extract metadata /audio features from a song ###

In [0]:
# extract audio features for each song
def audio_features(song_info, song_name, track_id, album_metadata) :    
    spotify_song = {}
  
    #add song metadata
    spotify_song['user_id']= song_info[0]
    spotify_song['age']= song_info[1]
    spotify_song['gender']= song_info[2]
    spotify_song['mood']= song_info[3]
    spotify_song['location']= song_info[4]
    spotify_song['activity']= song_info[5]
    spotify_song['period']= song_info[6]
                                       
    features = sp.audio_features(track_id)
    if (features is not None) and (features[0] is not None):    
      print(features)
      spotify_song['id'] = features[0]['id']
      #Append to relevant key-value
      spotify_song['acousticness'] = features[0]['acousticness']
      spotify_song['danceability'] = features[0]['danceability']
      spotify_song['energy'] = features[0]['energy']
      spotify_song['instrumentalness'] = features[0]['instrumentalness']
      spotify_song['liveness'] = features[0]['liveness']
      spotify_song['loudness'] = features[0]['loudness']
      spotify_song['speechiness'] = features[0]['speechiness']
      spotify_song['tempo'] = features[0]['tempo']
      spotify_song['valence'] = features[0]['valence']
      #popularity is stored elsewhere
      pop = sp.track(track_id)#track['id'])
      spotify_song['popularity'] = pop['popularity']
      print(album_metadata)
      spotify_song['release_date'] = album_metadata['album']['release_date']
      spotify_song['artist_name'] = album_metadata['artists'][0]['name']    
      spotify_song['song_name'] = pop['name']       

    return spotify_song

def add_empty_row(song_info, song_name):
    spotify_song = {}

    #add song metadata
    spotify_song['user_id']= song_info[0]
    spotify_song['age']= song_info[1]
    spotify_song['gender']= song_info[2]
    spotify_song['mood']= song_info[3]
    spotify_song['location']= song_info[4]
    spotify_song['activity']= song_info[5]
    spotify_song['period']= song_info[6]
    
    spotify_song['id']=pd.NaT
    spotify_song['acousticness'] = pd.NaT
    spotify_song['danceability'] = pd.NaT
    spotify_song['energy'] = pd.NaT
    spotify_song['instrumentalness'] = pd.NaT
    spotify_song['liveness'] = pd.NaT
    spotify_song['loudness'] = pd.NaT
    spotify_song['speechiness'] = pd.NaT
    spotify_song['tempo'] = pd.NaT
    spotify_song['valence'] = pd.NaT        
    spotify_song['popularity'] = pd.NaT
    spotify_song['release_date'] = pd.NaT
    spotify_song['artist_name'] = song_name.split(':')[1]
    spotify_song['song_name'] = song_name.split(':')[0]

    return spotify_song


### Search spotify for the songs in my list ###

#### Specify user lists ####

In [11]:
#for test purposes test search to Spotify
artistName="Blackmore*"
trackName='Where are we going from here'
q = "{0} artist:{1}".format(trackName,artistName)
sp.search(q)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Where+are+we+going+from+here+artist%3ABlackmore*&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5BnNG7QQQDTNd43ozx5y13'},
       'href': 'https://api.spotify.com/v1/artists/5BnNG7QQQDTNd43ozx5y13',
       'id': '5BnNG7QQQDTNd43ozx5y13',
       'name': "Blackmore's Night",
       'type': 'artist',
       'uri': 'spotify:artist:5BnNG7QQQDTNd43ozx5y13'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS

In [0]:
song_list = final_data

In [0]:
 songs_found = []

for i,row in song_list.iterrows():  #chosen track  
  song = row['song'].strip()+':'+row['artist'].strip()
  result = sp.search(song) #search query
  song_info = [row['user_id'], row['age'], row['gender'], row['mood'], row['location'], row['activity'], row['period']]
  if len(result['tracks']['items'])>0:
    print(str(row['user_id']) + ':'+ song)
    track_id = result['tracks']['items'][0]['id']
    #print(track_id, result['tracks']['items'][0],result['tracks']['items'][0] )
    #print(sp.audio_features([track_id]))
    songs_found.append(audio_features(song_info, song, track_id, result['tracks']['items'][0]))
  else:
    print('song {0} not found'.format(song))
    songs_found.append(add_empty_row(song_info, song))
    #Append to relevant key-value
    
print('added {0} tracks'.format(len(songs_found)))

1:My Angel:Stive Morgan
[{'danceability': 0.517, 'energy': 0.671, 'key': 11, 'loudness': -10.072, 'mode': 0, 'speechiness': 0.0279, 'acousticness': 0.143, 'instrumentalness': 0.392, 'liveness': 0.0974, 'valence': 0.862, 'tempo': 188.021, 'type': 'audio_features', 'id': '6x4QOwELkBmymbbm3Q2gvz', 'uri': 'spotify:track:6x4QOwELkBmymbbm3Q2gvz', 'track_href': 'https://api.spotify.com/v1/tracks/6x4QOwELkBmymbbm3Q2gvz', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6x4QOwELkBmymbbm3Q2gvz', 'duration_ms': 347439, 'time_signature': 4}]
{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5gCBbAjdGfXKthsKauWaYo'}, 'href': 'https://api.spotify.com/v1/artists/5gCBbAjdGfXKthsKauWaYo', 'id': '5gCBbAjdGfXKthsKauWaYo', 'name': 'Stive Morgan', 'type': 'artist', 'uri': 'spotify:artist:5gCBbAjdGfXKthsKauWaYo'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/6YgTYDvw0YcQixcb4yS305'}, 'href': 'https://api.spotify.com/v1/ar

In [0]:
df = pd.io.json.json_normalize(songs_found)
#display(list(songs_found))
print(len(df))
df.to_excel('spotify_metadata_playlist.xlsx')
df.head(20)

703


,acousticness,activity,age,artist_name,danceability,energy,gender,id,instrumentalness,liveness,...,loudness,mood,period,popularity,release_date,song_name,speechiness,tempo,user_id,valence
0,0.143,studying,44,Stive Morgan,0.517,0.671,female,6x4QOwELkBmymbbm3Q2gvz,0.392,0.0974,...,-10.072,pleased,evening,19,2019-04-25,My Angel (Piano Theme 2019),0.0279,188.021,1,0.862
1,0.407,working,44,The Rasmus,0.544,0.463,female,4khu91lkXFEebpfrUs3f81,0,0.0748,...,-6.832,pleased,evening,43,2009-11-09,October & April,0.0294,131.842,1,0.282
2,0.696,working,44,R Armando Morabito,0.553,0.528,female,7f7f1GcRnsa4EZ2VZ1ZcsV,0.00108,0.104,...,-12.852,pleased,evening,33,2014-12-23,One More Time (feat. Malukah & Tina Guo),0.0306,143.937,1,0.0952
3,0.0375,studying,44,Hilmer Rose,0.647,0.675,female,2z5XDfumoRcIZLQL9i6YhX,0,0.0732,...,-6.692,relaxed,morning,7,2016-10-20,Let Me Love You - Reprise DJ Snake Feat Justin...,0.117,109.93,1,0.909
4,0.0275,studying,44,Harry Styles,0.516,0.595,female,5Ohxk2dO5COHF1krpoPigN,0,0.109,...,-4.63,relaxed,morning,78,2017-05-12,Sign of the Times,0.0313,119.972,1,0.222
5,0.142,studying,44,Miley Cyrus,0.459,0.545,female,7ywzEdEuVG1lsjsRCGJfLy,0,0.105,...,-4.874,relaxed,morning,60,2009-01-01,When I Look At You,0.0282,137.941,1,0.194
6,0.123,working,44,Alan Walker,0.478,0.633,female,0nrWZWkJM62ftQJSI8fDc0,0,0.41,...,-6.19,relaxed,afternoon,67,2018-12-14,Diamond Heart,0.0345,89.909,1,0.287
7,0.0641,working,44,Titiyo,0.645,0.787,female,7GmHOAdriOnI9s5S79GAUr,0.513,0.341,...,-4.707,relaxed,afternoon,54,2001-01-01,Come Along,0.0274,84.116,1,0.906
8,0.143,working,44,Stive Morgan,0.517,0.671,female,6x4QOwELkBmymbbm3Q2gvz,0.392,0.0974,...,-10.072,nervous,afternoon,19,2019-04-25,My Angel (Piano Theme 2019),0.0279,188.021,1,0.862
9,0.287,amusing,44,Andy Moor,0.441,0.664,female,4oEFasfKUTARKcju6qkJdG,1e-05,0.141,...,-8.958,sad,evening,31,2013-07-05,Trespass - Masoud Chill Out Mix,0.0424,132.136,1,0.201


In [0]:
final_data['gender'].value_counts()
final_data['period'].value_counts()
final_data['mood'].value_counts()
final_data['activity'].value_counts()

working        362
amusing        141
commuting       44
studying        38
working out     29
other            9
Name: activity, dtype: int64